In [4]:
import os
import shutil

folder_path = '/kaggle/working/'

for item in os.listdir(folder_path):
    item_path = os.path.join(folder_path, item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)  # remove file
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)  # remove directory and its contents
    except Exception as e:
        print(f'Failed to delete {item_path}. Reason: {e}')

print(f"Contents of {folder_path} cleared.")

Contents of /kaggle/working/ cleared.


In [5]:
import os
import zipfile
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

from torchvision import transforms

import torch
from torch.utils.data import Dataset
import os

import kornia.augmentation as K

label_map = {"Alluvial soil": 0, "Black Soil": 1, "Clay soil": 2, "Red soil": 3}

class SoilDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):  # Accept DataFrame
        self.data = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.data.iloc[index, 0])
        image = Image.open(img_path).convert("RGB")
        label = label_map[self.data.iloc[index, 1]]
        if self.transform:
            image = self.transform(image)
        return image, label

from torch.utils.data import DataLoader

import torchvision.models as models
import torch.nn as nn

# Training transforms
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#Augmentation performed on gpu for faster training
gpu_augmentations = nn.Sequential(
    K.RandomHorizontalFlip(p=0.5),
    # training for angled images
    K.RandomRotation(degrees=30),
    #train for blurry images
    K.RandomGaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0), p=0.5),
).to("cuda")

# Validation transforms (no augmentation)
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_normalize = nn.Sequential(
    # K.Normalize(
    #     mean=torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1),
    #     std=torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
    # )
).to("cuda")

from PIL import Image
import os

def resize_and_centercrop224(img):
        original_width, original_height = img.size
        target_width, target_height = (224, 224)

        # Calculate aspect ratios
        original_aspect = original_width / original_height
        target_aspect = target_width / target_height

        if original_aspect > target_aspect:
            # Original image is wider than target: Resize based on height
            new_height = target_height
            new_width = int(new_height * original_aspect)
        else:
            # Original image is taller than target (or same aspect): Resize based on width
            new_width = target_width
            new_height = int(new_width / original_aspect)

        # Resize the image
        img_resized = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # Calculate coordinates for centercropping
        left = (new_width - target_width) / 2
        top = (new_height - target_height) / 2
        right = (new_width + target_width) / 2
        bottom = (new_height + target_height) / 2

        # Crop the image
        img_cropped = img_resized.crop((left, top, right, bottom))

        return img_cropped
        
def preprocess_images(input_dir, output_dir, size=224):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        img = Image.open(os.path.join(input_dir, filename)).convert("RGB")
        img = resize_and_centercrop224(img)
        img.save(os.path.join(output_dir, filename))

#resize only one time instead of transforms.resize in every fetch cycle
preprocess_images("/kaggle/input/c/soil-classification/soil_classification-2025/train", "train_resized")
print("images saved")

df = pd.read_csv("/kaggle/input/c/soil-classification/soil_classification-2025/train_labels.csv")

#train-test split for validation dataset
train_df, val_df = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df["soil_type"],
    random_state=42
)

# Training dataset
train_dataset = SoilDataset(
    dataframe=train_df,
    root_dir="train_resized",
    transform=train_transform
)

# Validation dataset
val_dataset = SoilDataset(
    dataframe=val_df,
    root_dir="train_resized",
    transform=val_transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True, persistent_workers=True)
# No shuffle for val
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)

model = models.resnet18(pretrained=True) # for small dataset
model = nn.DataParallel(model) # use both gpus
model = model.to("cuda")

images saved


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNe

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_val_loss = float("inf")
patience = 10 # early stopping after 10 worse val losses
epochs_no_improve = 0

for epoch in range(50):
    model.train()
    running_train_loss = 0.0
    
    # Training phase
    for images, labels in train_loader:
        images, labels = images.to("cuda"), labels.to("cuda")
        images = gpu_augmentations(images)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_train_loss += loss.item()
    
    # Validation phase
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to("cuda"), labels.to("cuda")
            images = val_normalize(images)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
    
    # Calculate average losses
    avg_train_loss = running_train_loss / len(train_loader)
    avg_val_loss = running_val_loss / len(val_loader)
    
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.2f}, Val Loss: {avg_val_loss:.2f}")
    
    # Early stopping check (based on validation loss)
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        #save best model parameters for generating submission.csv
        torch.save(model.state_dict(), "best_model.pth")
        print("saved best_model.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered!")
            break

Epoch 1, Train Loss: 1.78, Val Loss: 6.21
saved best_model.pth
Epoch 2, Train Loss: 0.62, Val Loss: 2.91
saved best_model.pth
Epoch 3, Train Loss: 0.28, Val Loss: 0.30
saved best_model.pth
Epoch 4, Train Loss: 0.42, Val Loss: 0.28
saved best_model.pth
Epoch 5, Train Loss: 0.45, Val Loss: 0.76
Epoch 6, Train Loss: 0.23, Val Loss: 0.22
saved best_model.pth
Epoch 7, Train Loss: 0.22, Val Loss: 0.20
saved best_model.pth
Epoch 8, Train Loss: 0.18, Val Loss: 0.20
saved best_model.pth
Epoch 9, Train Loss: 0.15, Val Loss: 0.17
saved best_model.pth
Epoch 10, Train Loss: 0.17, Val Loss: 0.17
Epoch 11, Train Loss: 0.26, Val Loss: 0.23
Epoch 12, Train Loss: 0.20, Val Loss: 0.28
Epoch 13, Train Loss: 0.16, Val Loss: 0.19
Epoch 14, Train Loss: 0.18, Val Loss: 0.23
Epoch 15, Train Loss: 0.18, Val Loss: 0.21
Epoch 16, Train Loss: 0.16, Val Loss: 0.34
Epoch 17, Train Loss: 0.19, Val Loss: 0.25
Epoch 18, Train Loss: 0.16, Val Loss: 0.25
Epoch 19, Train Loss: 0.12, Val Loss: 0.16
saved best_model.pth
Epo

In [7]:
from sklearn.metrics import f1_score

model.load_state_dict(torch.load("/kaggle/input/best-model-part-1/best_model.pth"))
model.eval()
y_true, y_pred = [], []

# Validation dataset
dataset = SoilDataset(
    dataframe=df,
    root_dir="train_resized",
    transform=val_transform
)

val_loader = DataLoader(dataset, batch_size=64, shuffle=False)

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to("cuda")
        images = val_normalize(images)
        outputs = model(images)
        preds = torch.argmax(outputs, 1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(labels.numpy())

f1s = f1_score(y_true, y_pred, average=None)
print("F1 Scores for each class:", f1s)
print("Average F1 (Macro):", np.mean(f1s))

F1 Scores for each class: [0.98265896 0.98717949 0.96350365 0.99810247]
Average F1 (Macro): 0.9828611407863161


In [8]:
submission = []

val_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model.eval()
for filename in sorted(os.listdir("/kaggle/input/c/soil-classification/soil_classification-2025/test")):
    img = Image.open(os.path.join("/kaggle/input/c/soil-classification/soil_classification-2025/test", filename)).convert("RGB")
    img = val_normalize(val_transform(img)).unsqueeze(0).to("cuda")
    output = model(img)
    pred = torch.argmax(output, 1).item()
    label = list(label_map.keys())[list(label_map.values()).index(pred)]
    submission.append((filename, label))

pd.DataFrame(submission, columns=["filename", "label"]).to_csv("submission.csv", index=False)
print("submission.csv generated")

submission.csv generated
